In [1]:
import pandas as pd

# 1. 데이터 불러오기
df = pd.read_csv('./data/working/tpss_bcycl_od_statnhm_202406_통합_w_자치구.csv')

In [2]:
# 2. 날짜 형식 변환 및 컬럼 생성 (주간, 시간대)
df['기준_날짜'] = pd.to_datetime(df['기준_날짜'], format='%Y%m%d')
df['주간'] = df['기준_날짜'].dt.dayofweek.apply(lambda x: '주말' if x >= 5 else '평일')
df['시간대'] = df['기준_시간대'] // 100

In [3]:
# 3. 중구에 위치한 모든 대여소의 고유 목록 생성
junggu_stations_start = df[df['시작_대여소_자치구'] == '중구'][['시작_대여소_ID', '시작_대여소명']].rename(columns={'시작_대여소_ID': '대여소_ID', '시작_대여소명': '대여소명'})
junggu_stations_end = df[df['종료_대여소_자치구'] == '중구'][['종료_대여소_ID', '종료_대여소명']].rename(columns={'종료_대여소_ID': '대여소_ID', '종료_대여소명': '대여소명'})
all_junggu_stations = pd.concat([junggu_stations_start, junggu_stations_end]).drop_duplicates().reset_index(drop=True)

In [4]:
# 4. '동일 대여소 반납' 건수 계산
same_station_df = df[(df['시작_대여소_ID'] == df['종료_대여소_ID']) & (df['시작_대여소_자치구'] == '중구')]
same_station_counts = same_station_df.groupby(['시작_대여소_ID', '주간', '시간대']).size().reset_index(name='동일_대여소_반납')

In [5]:
# 5. '유출(Outflow)' 건수 계산 (중구 -> 타 자치구)
outflow_df = df[(df['시작_대여소_자치구'] == '중구') & (df['종료_대여소_자치구'] != '중구')]
outflow_counts = outflow_df.groupby(['시작_대여소_ID', '주간', '시간대']).size().reset_index(name='유출_건수')

In [6]:
# 6. '유입(Inflow)' 건수 계산 (타 자치구 -> 중구)
inflow_df = df[(df['종료_대여소_자치구'] == '중구') & (df['시작_대여소_자치구'] != '중구')]
inflow_counts = inflow_df.groupby(['종료_대여소_ID', '주간', '시간대']).size().reset_index(name='유입_건수')

In [7]:
# 7. '중구 내 이동' 건수 계산 (중구 -> 중구, 다른 대여소)
junggu_internal_df = df[(df['시작_대여소_자치구'] == '중구') & (df['종료_대여소_자치구'] == '중구') & (df['시작_대여소_ID'] != df['종료_대여소_ID'])]
junggu_internal_counts = junggu_internal_df.groupby(['시작_대여소_ID', '주간', '시간대']).size().reset_index(name='중구_내_이동_건수')

In [8]:
# 8. 분석을 위한 기본 데이터 틀 생성
hours = pd.DataFrame({'시간대': range(24)})
week_types = pd.DataFrame({'주간': ['평일', '주말']})
base_df = pd.merge(all_junggu_stations, week_types, how='cross')
base_df = pd.merge(base_df, hours, how='cross')

In [9]:
# 9. 계산된 결과들을 기본 틀에 병합
summary_final_df = pd.merge(base_df, same_station_counts.rename(columns={'시작_대여소_ID': '대여소_ID'}), on=['대여소_ID', '주간', '시간대'], how='left')
summary_final_df = pd.merge(summary_final_df, outflow_counts.rename(columns={'시작_대여소_ID': '대여소_ID'}), on=['대여소_ID', '주간', '시간대'], how='left')
summary_final_df = pd.merge(summary_final_df, inflow_counts.rename(columns={'종료_대여소_ID': '대여소_ID'}), on=['대여소_ID', '주간', '시간대'], how='left')
summary_final_df = pd.merge(summary_final_df, junggu_internal_counts.rename(columns={'시작_대여소_ID': '대여소_ID'}), on=['대여소_ID', '주간', '시간대'], how='left')

In [21]:
# 10. 최종 데이터 정리
count_cols = ['동일_대여소_반납', '유출_건수', '유입_건수', '중구_내_이동_건수']
summary_final_df[count_cols] = summary_final_df[count_cols].fillna(0).astype(int)
summary_final_df = summary_final_df.sort_values(by=['대여소_ID', '주간', '시간대']).reset_index(drop=True)

In [25]:
# 11. CSV 파일로 저장
summary_final_df.to_csv('./data/columns중구/중구컬럼5-6.csv', index=False)

print("분석 완료! '중구 내 이동'을 포함한 최종 결과가 '중구컬럼5-6.csv' 파일로 생성되었습니다.")
print(summary_final_df.head())

분석 완료! '중구 내 이동'을 포함한 최종 결과가 '중구컬럼5-6.csv' 파일로 생성되었습니다.
   대여소_ID       대여소명  주간  시간대  동일_대여소_반납  유출_건수  유입_건수  중구_내_이동_건수
0  ST-116  소공동_002_1  주말    0          0      1      3           0
1  ST-116  소공동_002_1  주말    1          0      2      1           0
2  ST-116  소공동_002_1  주말    2          0      5      1           2
3  ST-116  소공동_002_1  주말    3          0      2      0           0
4  ST-116  소공동_002_1  주말    4          0      0      0           0
